In [ ]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataFrames
using Plots

include(srcdir("nodeconstructor.jl"))
include(srcdir("pv_module.jl"))

include(srcdir("env.jl"));

## The PV-Array

This notebook will explain the PV arrays structurally and how they were implemented.

As a basis serves the [paper](https://doi.org/10.1016/j.nrjag.2014.04.001) which explains the physical characteristics of a PV module in more detail. In order to avoid further calculations at runtime, a simpler approximation is used, the ideal single diode model. This neglects the parallel and series resistance of the PV module.

In the following we define a mutable struct with the most important data of a PV module, most of the data are given by the data sheets.

In [ ]:
PV_mod = PV_module(); # For this example we use the default values

As a rule, many of the modules are initially connected in series, especially in large systems, in order to increase the voltage. Further modules can then be added in parallel to further increase the current.

We therefore define another mutable struc, which inherits the properties of the PV module and specifies how many modules are to be created in series and then again in parallel to it.

For real applications, this often leads to problems, since shadowing of individual cells (!!!) can result in a large drop in power. Therefore, in practice, countermeasures are often taken, such as the integration of bypass diodes.

In [ ]:
PV_arr = PV_array(;pv_module=PV_mod)

Now let us take a look  characteristics for different irradiations. Therefore, we define a function that gives us the current as a function of voltage, irradiation and temperature. In addition, a function should also output the voltage as a function of current, irradiation and temperature, since this will later be interesting for our application in DARE.

### Funktion for module

For the purpose of this notebook, the functions here are defined vectorially so that they can be evaluated directly for multiple irradiances and voltages.

The functions that are later used in the DARE environment return only a scalar value.

In [ ]:
function get_I_vec(self::PV_module, V, G, T)
    
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G./self.G_ref*(self.I_ph_ref + self.mu_sc * dT).* ones(length(V))'
        return I_ph
    end;

    function I_diode(self::PV_module, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0*(exp.(V/(self.ni*self.N_cell*V_T)).-1).*ones(5)'
        return I_d
    end;
    
    I = (I_photo(self, G, T)' - I_diode(self, V, G, T))
    return I
end


In [ ]:
# Not working!

function get_V_vec(self::PV_module, I, G, T)
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)* ones(length(I))';
        return I_ph
    end;
    
#     I = maximum.([0,I])
    
    res = (I_photo(self, G, T)-I)
    
    dT = self.T_0 + T
    V_T = self.k*dT/self.q
    
    if res <= 0
        V=0
    else
        V = self.ni*self.N_cell*V_T*(log((res)/self.I_0)+1)
    end
    
    return V
end

## Characteristics for PV modules

Here the values are defined for which the modules are to be evaluated.

In [ ]:
T = 25
G = collect(200:200:1000)
V = collect(0:0.1:30);

In [ ]:
i = get_I_vec(PV_mod, V, G, T);

In [ ]:
labels = ["200 W/m^2" "400 W/m^2" "600 W/m^2" "800 W/m^2" "1000 W/m^2"];
plot(V, i, xlim=(0,30),ylim=(0,5),title="PV module - I(V)", label=labels, xlabel="V", ylabel="I")

In [ ]:
V_ = V.*ones(5)';
P = i .* V_;

In [ ]:
plot(V_, P, xlim=(0,30), ylim=(0,100),title="PV module - P(V)", label=labels, xlabel="V", ylabel="P")

In [ ]:
plot(i, V, xlim=(0,5), ylim=(0,30),title="PV module - V(I)", label=labels, xlabel="I", ylabel="V")

In [ ]:
plot(i, P, xlim=(0,5), ylim=(0,80),title="PV module - P(I)", label=labels, xlabel="I", ylabel="P")

For the characteristics depending on x, the axis have been swapped here. When plotting with the explicit function, there is a problem with the logarithm, which must not be less than zero. Therefore, the range of values must be very fine tuned. But since we can see in the upper two plots how steep the function is, this is very difficult.

In [ ]:
# V = get_V(PV_mod, I, 1000, T);

### Funktion for arrays

In [ ]:
function get_I_vec(pv_arr::PV_array, V, G, T)
    
    self = pv_arr.pv_module
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G./self.G_ref*(self.I_ph_ref + self.mu_sc * dT).* ones(length(V))'
        return I_ph
    end;

    function I_diode(self::PV_module, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0*(exp.(V/(self.ni*self.N_cell*pv_arr.serial*V_T)).-1).*ones(5)'
        return I_d
    end;
    
    I = (I_photo(self, G, T)' - I_diode(self, V, G, T)) * pv_arr.parallel
    return I
end

In [ ]:
# Not working!

function get_V_vec(pv_arr::PV_array, I, G, T)
    
    self = pv_arr.pv_module
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)
        return I_ph
    end;
    
    I = maximum([0,I])
    res = (I_photo(self, G, T)-(I/pv_arr.parallel))
    
    dT = self.T_0 + T
    V_T = self.k*dT/self.q
    
    if res <= 0
        V=0
    else
        V = self.ni*self.N_cell*pv_arr.serial*V_T*(log((res)/self.I_0)+1)
    end
    
    return V
end

### Select Test point

Here we use the functions that are also stored in the DARE to evaluate the characteristic curves at selected points.

In [ ]:
v_test = 200
i_test = get_I(PV_arr, v_test, 1000, 27)

In [ ]:
i_test2 = 13.951122339665764
v_test2 = get_V(PV_arr, i_test2, 1000, 27)

## Characteristics for PV arrays

In [ ]:
T = 25
G = collect(200:200:1000)
V = collect(0:1:500);

In [ ]:
i = get_I_vec(PV_arr, V, G, T);

In [ ]:
plot(V, i, xlim=(0,350),ylim=(0,20),title="PV array - I(V)", label=labels, xlabel="V", ylabel="I")
scatter!([v_test], [i_test], color="blue", markershape=:star5, label="Test")

In [ ]:
V_ = V.*ones(5)';
P = i .* V_;

In [ ]:
plot(V_, P, xlim=(0,300), ylim=(0,3500),title="PV array - P(V)", label=labels, xlabel="V", ylabel="P")

In [ ]:
plot(i, V_, xlim=(0,30), ylim=(0,300),title="PV array - V(I)", label=labels, xlabel="I", ylabel="V")
scatter!([i_test2],[v_test2], color="blue", markershape=:star5, label="Test")

Here you can see that the test point is not exactly on the line. This is due to the numerical inaccuracy. Since we are on the steep part, small deviations lead to larger shifts along the y-axis.

In [ ]:
plot(i, P, xlim=(0,20), ylim=(0,3500),title="PV array - V(I)", label=labels, xlabel="I", ylabel="P")

## With R_s and R_p

Here is the code for the iterative determination of the characteristic curve via newton raphson. The parallel and series resistance of the PV module are now taken into account.

In [ ]:
dT = 273 + T
V_T = k*dT/q   
I_ph = G./G_ref*(I_ph_ref + mu_sc * dT)*ones(length(V))';
I_d = I_0*(exp.(V/(ni*N_cell*V_T)).-1).*ones(5)';

I_init = 1 .*ones(5)

R_s = 0.45                 # 
R_p = 310.0248             # 
        

function get_I(G, V)
    
    I_ph = ones(length(V)) * G/G_ref*(I_ph_ref + mu_sc * dT);
    delta = 0.00000000001
    
    function I_d(V, I)        
        a = ni*N_cell*V_T
        R_s = 0.45 
        return I_0*(exp.(V.+R_s.*I/(a)).-1)
    end

    function I_p(V, I)
        R_p = 310.0248
        R_s = 0.45 
        return V.+R_s.*I/R_p
    end
    
    function f_dot(V,I)
        a = ni*N_cell*V_T
        R_p = 310.0248
        R_s = 0.45 
        return -I_0*1/a*exp.(V.+R_s.*I/(a)).-1/R_p
    end
    
    I = 3*ones(length(V))
    I_= zeros(length(V))
    
    while true
        
        I_ = I - (I_ph - I_d(V,I) - I_p(V,I))./f_dot(V,I)
        
        check = sum(broadcast(abs, (I_-I)) .< delta)
        if check == length(V)
            break
        end
        I = I_
    end
            
    return I_
end
    